## AWS S3 bucket integraion with Databricks

Importing the necessary modules from spark and urllib to encode sensitive information. Creating the source url with the necessary credentias and performing the integration.

In [0]:
from pyspark.sql.functions import *
import urllib

def mount_s3_bucket():

    try:
        delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"
        aws_keys_df = spark.read.format("delta").load(delta_table_path)

        ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
        SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
        ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

        AWS_S3_BUCKET = "user-12a740a19697-bucket/topics"
        MOUNT_NAME = "/mnt/12a740a19697_bucket"
        SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)

        dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)
    except:
        print(f'Directory already mounted: {MOUNT_NAME}')

mount_s3_bucket()

Directory already mounted: /mnt/12a740a19697_bucket


The integration has been executed effectively, and by accessing the content of the bucket, the information stored on the AWS S3 bucket becomes apparent.

In [0]:
display(dbutils.fs.ls("/mnt/12a740a19697_bucket/"))

path,name,size,modificationTime
dbfs:/mnt/12a740a19697_bucket/12a740a19697.geo/,12a740a19697.geo/,0,1706628315215
dbfs:/mnt/12a740a19697_bucket/12a740a19697.pin/,12a740a19697.pin/,0,1706628315215
dbfs:/mnt/12a740a19697_bucket/12a740a19697.user/,12a740a19697.user/,0,1706628315215


In [0]:
%sql
SET spark.databricks.delta.formatCheck.enabled=false

key,value
spark.databricks.delta.formatCheck.enabled,false


In [0]:
def make_df_tables(df_name, topic):

    file_location = f"/mnt/12a740a19697_bucket/12a740a19697.{topic}/partition=0/*.json"
    file_type = "json"
    infer_schema = "true"

    df_name = spark.read.format(file_type).option("inferSchema", infer_schema).load(file_location)
    return df_name   
